<a href="https://colab.research.google.com/github/FaithXiaonuo/NeuralNetworks/blob/Neural_Networks/Dense_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instructions**
For this task, we need to build our own fully connected neural network without the help of additional python packages (such as numpy).
There will be two parts. First, we build a Keras model to train the MNIST data set and save the parameters. Second, we use our own neural network to rebuild the neural network based on the saved parameters.

# **part 1**
In this part, we will use the keras model to build a neural network. Train the model so that the accuracy of training and verification can reach more than 97% and 93%. Save the parameters, which will be used in Part 2, including weights and biases.

In [1]:
# Import TensorFlow into your program:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, ReLU
from tensorflow.keras import Model

from keras.datasets import mnist

from google.colab import files
import h5py

In [2]:
# Load and prepare the MNIST dataset.
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# Use tf.data to batch and shuffle the dataset:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [3]:
# Build the tf.keras model using the Keras model subclassing API:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(28, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [4]:
# Choose an optimizer and loss function for training:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Select metrics to measure the loss and the accuracy of the model. 
# These metrics accumulate the values over epochs and then print the overall result.
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [5]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [6]:
EPOCHS = 10

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
      f'Epoch {epoch + 1}, '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result() * 100}, '
     f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.382997065782547, Accuracy: 89.31666564941406, Test Loss: 0.2404157519340515, Test Accuracy: 92.91999816894531
Epoch 2, Loss: 0.21424515545368195, Accuracy: 93.80833435058594, Test Loss: 0.189997136592865, Test Accuracy: 94.2300033569336
Epoch 3, Loss: 0.1723167598247528, Accuracy: 95.0250015258789, Test Loss: 0.1650402694940567, Test Accuracy: 95.04000091552734
Epoch 4, Loss: 0.14684751629829407, Accuracy: 95.75499725341797, Test Loss: 0.1508215367794037, Test Accuracy: 95.51000213623047
Epoch 5, Loss: 0.12937599420547485, Accuracy: 96.24333190917969, Test Loss: 0.14183661341667175, Test Accuracy: 95.6300048828125
Epoch 6, Loss: 0.11640485376119614, Accuracy: 96.61166381835938, Test Loss: 0.13181443512439728, Test Accuracy: 95.92000579833984
Epoch 7, Loss: 0.10591881722211838, Accuracy: 96.95166778564453, Test Loss: 0.13015875220298767, Test Accuracy: 95.9000015258789
Epoch 8, Loss: 0.09730491042137146, Accuracy: 97.19667053222656, Test Loss: 0.12444835901260376, Test 

In [7]:
# Save the weights
model.save_weights('/content/my_model.h5')
# files.download('/content/my_model.h5')

In [8]:
# Restore the weights
model2 = MyModel()
model2.build((32, 28, 28, 1))
model2.compile(
    optimizer=optimizer,
    loss=loss_object,
    metrics=['accuracy']
)
model2.load_weights('/content/my_model.h5')
model2.evaluate(x_train, y_train)
model2.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1219 - accuracy: 0.9638


[0.12194637209177017, 0.9638000130653381]

# **Part 2**
In this part, we use our own neural network to reconstruct the neural network based on the parameter file. 

In [9]:
# load weights data from .h5 file
load_weights = h5py.File('/content/my_model.h5')
# convert weights and biases into lists
load_w_d0 = load_weights['dense']['my_model']['dense']['kernel:0'][:]
load_w_d1 = load_weights['dense_1']['my_model']['dense_1']['kernel:0'][:]

load_b_d0 = load_weights['dense']['my_model']['dense']['bias:0'][:]
load_b_d1 = load_weights['dense_1']['my_model']['dense_1']['bias:0'][:]

In [10]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()
train_x, test_x = train_x / 255.0, test_x / 255.0
train_x = train_x[..., tf.newaxis].astype("float32")
test_x = test_x[..., tf.newaxis].astype("float32")
train_x = [train_x[i:i+32] for i in range(int(len(train_x) / 32))]
train_y = [train_y[i:i+32] for i in range(int(len(train_y) / 32))]
test_x = [test_x[i:i+32] for i in range(int(len(test_x) / 32))]
test_y = [test_y[i:i+32] for i in range(int(len(test_y) / 32))]

In [11]:
# reimplemented the layer function, including flatten, ReLU, Math of Matrix(Dot, Plus)
def flatten(data):
    output = [[[item for sublist in batch for filter in sublist for item in filter]] for batch in data]
    return output

In [12]:
def myReLUforDense(input):
    input = [[[0 if input[i][0][k] < 0 else input[i][0][k] for k in range(len(input[i][0]))]] for i in range(len(input))]
    return input

In [13]:
def myMax(input):
    output = []
    for i in range(len(input)):
        max = 0
        for j in range(1, len(input[i][0])):
            if input[i][0][max] < input[i][0][j]:
                max = j
        output.append(max)
    return output

In [14]:
def myDot(MA, MB):
    output = [[sum(a * b for a, b in zip(A_row, B_col)) 
                              for B_col in zip(*MB)]
                              for A_row in MA]
    return output

def myPlus(A, B):
    output = [[A[0][i] + B[i] for i in range(len(A[0]))]]
    return output

In [15]:
# my reimplemented Dense 
class Dense():
    def __init__(self, weights, biases):
        super(Dense, self).__init__()
        self.weights = weights
        self.biases = biases
    
    def forward(self, inputs):
        return [myPlus(myDot(input, self.weights), self.biases) for input in inputs]

In [16]:
# my rewrite model
def myNewModel(input, dense0, dense1):
    input = flatten(input)
    input = dense0.forward(input)
    input = myReLUforDense(input)
    input = dense1.forward(input)
    input = myMax(input)
    return input

In [17]:
acc_train = 0
lgh_train = 0
acc_test = 0
lgh_test = 0
dense0 = Dense(load_w_d0, load_b_d0)
dense1 = Dense(load_w_d1, load_b_d1)

for i in range(len(train_y)): 
    output = []
    diff = 0
    out = myNewModel(train_x[i], dense0, dense1)
    for index in range(len(out)):
        if out[index] != train_y[i][index]:
            diff += 1
    acc = 1 - diff/len(labels)
    acc_train += acc
    lgh_train += 1

for i in range(len(test_y)): 
    output = []
    diff = 0
    out = myNewModel(test_x[i], dense0, dense1)
    for index in range(len(out)):
        if out[index] != test_y[i][index]:
            diff += 1
    acc = 1 - diff/len(labels)
    acc_test += acc
    lgh_test += 1

print("train_accuracy:" + str(acc_train/lgh_train * 100) + ". test_accuracy:" + str(acc_test/lgh_test * 100))

batch: 20 acc: 1.0
batch: 40 acc: 1.0
batch: 60 acc: 1.0
batch: 80 acc: 1.0
batch: 100 acc: 1.0
batch: 120 acc: 0.9375
batch: 140 acc: 0.9375
batch: 160 acc: 1.0
batch: 180 acc: 1.0
batch: 200 acc: 0.96875
batch: 220 acc: 0.9375
batch: 240 acc: 0.9375
batch: 260 acc: 0.9375
batch: 280 acc: 0.96875
batch: 300 acc: 0.96875


ZeroDivisionError: ignored